In [134]:
import os
import heapq

class BinaryTreeNode:
    def __init__(self, value, freq):
        self.value = value
        self.freq = freq
        self.left = None
        self.right = None
        
    def __lt__(self, other):
        return self.freq < other.freq
    
    def __eq__(self, other):
        return self.freq == other.freq
    
class HuffmanCoding:
    def __init__(self):
        self.codes = {}
        self.reverse_codes = {}
        
    def __make_freq_dict(self, text):
        h = {}
        for i in text:
            h[i] = h.get(i, 0) + 1
            
        return h
        
    def __build_heap(self, freq_dict):
        heap = []
        
        for value in freq_dict:
            freq = freq_dict[value]
            node = BinaryTreeNode(value, freq)
            heapq.heappush(heap, node)
            
        return heap
            
    def __build_tree(self, heap):
        while len(heap) > 1:
            node1 = heapq.heappop(heap)
            node2 = heapq.heappop(heap)
            
            freq = node2.freq + node1.freq
            new = BinaryTreeNode('', freq)
            
            new.left = node2
            new.right = node1
            
            heapq.heappush(heap, new)
            
        root = heapq.heappop(heap)
        return root
    
    def __evaluate_codes(self, root, curr_bits=''):
        if not root:
            return
        
        if not root.left and not root.right:
            self.codes[root.value] = curr_bits
            self.reverse_codes[curr_bits] = root.value
            return
        
        self.__evaluate_codes(root.left, curr_bits+'0')
        self.__evaluate_codes(root.right, curr_bits+'1')
    
    def __encode(self, text):
        encoded_text = ''
        for i in text:
            encoded_text += self.codes[i]
            
        return encoded_text
            
    def __padding(self, encoded_text):
        padding_amount = 8 - (len(encoded_text)%8)
        encoded_text += '0'*padding_amount
        
        # when we will decode the text, first 8 bits will tell us that how much amount you did pad
        padded_info = '{0:08b}'.format(padding_amount)
        encoded_text = padded_info + encoded_text
        
        return encoded_text
        
    def __get_bytes_arr(self, padded_text):
        arr = []
        for i in range(0, len(padded_text), 8):
            _8bits = padded_text[i: i+8]
            arr.append(int(_8bits, 2))
            
        return arr
    
    def compress(self, path):
        # read the text from the file
        with open(path, 'r') as file:
            text = file.read()
            file.close()
            
        # make freq dict using text
        freq_dict = self.__make_freq_dict(text)
        
        # build heap of freq dict
        heap = self.__build_heap(freq_dict)
        
        # build binary tree using heap
        root = self.__build_tree(heap)
        
        # evaluate codes from tree
        self.__evaluate_codes(root)
        
        # encode the text using codes
        encoded_text = self.__encode(text)
        
        # pad the encoded text
        padded_text = self.__padding(encoded_text)
        
        # convert the coded text into bytes
        bytes_arr = self.__get_bytes_arr(padded_text)
        final_bytes = bytes(bytes_arr)
        
        # return the binary file
        file_name, extension = os.path.splitext(path)
        output_path = 'compressed_' + file_name + '.bin'
        with open(output_path, 'wb') as file:
            file.write(final_bytes)
            file.close()
            
        print('compressed')
    
    def __remove_padding(self, bin_data):
        padded_info = int(bin_data[: 8], 2)
         
        return bin_data[8: -padded_info]
    
    def __decode(self, encoded_text):
        decoded_text = ''
        code = ''
        
        for i in encoded_text:
            code += i
            if code in self.reverse_codes:
                decoded_text += self.reverse_codes[code]
                code = ''
                
        return decoded_text
    
    def decompress(self, path):
        with open(path, 'rb') as file:
            bin_data = ''
            byte = file.read(1)
            
            while byte:
                byte = ord(byte)
                _8bits = bin(byte)[2: ].rjust(8, '0')
                bin_data += _8bits
                
                byte = file.read(1)
                
            file.close()
            
        encoded_text = self.__remove_padding(bin_data)
        
        text = self.__decode(encoded_text)
        
        file_name, extension = os.path.splitext(path)
        output_path = 'decompressed_' + file_name + '.txt'
        with open(output_path, 'w') as file:
            file.write(text)
            file.close()
        
        print('decompressed')

In [135]:
h = HuffmanCoding()

In [136]:
h.compress('text_file_to_compress.txt')

compressed


In [138]:
h.decompress('compressed_text_file_to_compress.bin')

decompressed
